In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import warnings
import string

check_tg_nick = set(list(string.ascii_lowercase) + list('@'))

# База номеров и ников телочек
base = set()

# Скрипт переписки
s1 = 'Привет) понравилась) что хотела бы на сайте?'
#s2_prev = 'Честно говоря, любви всей жизни тут не ищу. приятное лайтовое общение, а дальше видно будет)'
s2 = 'Честно говоря, чего-то особо серьезного тут не ищу. приятное лайтовое общение, а дальше видно будет)'
s3 = 'пиши вотсап или тг, там удобнее'
# s4 = 'туда редко захожу'
# s5 = 'я не вижу такие сообщения, напиши текстом'

check_msg = 'Sorry, this type of message is not supported'

# stupid_social_networks = {'инст', 'вк', 'инста', 'инстаграм', 'вконтакте', 'инсте', 'инсту'}

n_seen_dialogues = 0
n_likes = 0

# Убираем ненужные warnings
warnings.filterwarnings("ignore")

s = Service('/Users/maksimveretennikov/Desktop/Coding/Project_Badoo/chromedriver')
driver = webdriver.Chrome(service = s)
driver.get('https://badoo.com/en-us/signin/?f=top')

def login():
    try:
        time.sleep(2)
        login_field = driver.find_element_by_css_selector('.text-field__input.text-field__input--ltr.js-signin-login')
        login_field.send_keys('veretennikov.maksi@yandex.ru')
        pass_field = driver.find_element_by_css_selector('.text-field__input.js-signin-password')
        pass_field.send_keys('192837465m')
        time.sleep(2)
        btn_login = driver.find_element_by_css_selector('.btn--block')
        btn_login.click()
        time.sleep(4)
    except Exception as e: 
        print('Не логинится( Ошибка:')
        #print(e)
        


def new_button():
    try:
        time.sleep(1)
        btn_new = driver.find_element_by_css_selector('.react-button.react-button--filled.react-button--md.react-button--color-primary.react-button--narrow')
        btn_new.click()
        time.sleep(1)
    except Exception as e: 
        print('Новая кнопка( Ошибка:')
#         print(e)

def skip():
    try:
        time.sleep(1)
        btn_skip = driver.find_element_by_css_selector('.js-chrome-pushes-deny')
        btn_skip.click()
        time.sleep(1)
    except Exception as e: 
        print('Не пропускает сообщение( Ошибка:')
        print(e)
        

def like(n):
    global n_likes
    for i in range(n):
        try:
            time.sleep(0.2)
            btn_like = driver.find_element_by_css_selector('.js-profile-header-vote-yes')
            btn_like.click()

            if n_seen_dialogues == 0 and n_likes == 0:
                skip()
            n_likes += 1
        except Exception as e: 
            click_close()
            time.sleep(0.5)
    time.sleep(2)
            

def analyse_messages(messages):
    time.sleep(1)
    try:
        for msg in messages:
#             if msg == check_msg:
#                 send_message(s5)
#                 return 0
            for w in msg.split():
                word = w.lower().replace('?', '').replace(',', '').replace('.', '').replace(')', '').replace('(', '').replace('-','').replace('+','')
                if word.isdigit() and len(word) >= 10:
                    return word
                elif word and word[0] in check_tg_nick:
                    return word
    except Exception as e: 
        print('Метод analyse_messages барахлит!( Ошибка:')
        print(e)
    return 0


def send_message(s):
    global n_seen_dialogues
    
    try:
        input_message_field = driver.find_element_by_css_selector('.messenger-tools__input')
        input_message_field.send_keys(s)

        time.sleep(0.5)
        btn_send_message = driver.find_element_by_css_selector('.js-send-message')
        btn_send_message.click()
        time.sleep(0.5)

        if n_seen_dialogues == 0 and n_likes == 0:
            skip()

        n_seen_dialogues += 1
    except Exception as e: 
        print('Не получилось отправить сообщение!( Ошибка:')
        #print(e)
        
def go_to_messages():
    try:
        time.sleep(1)
        url = '/messenger/open'
        message_link = driver.find_element_by_xpath('//a[@href="' + url + '"]')
        message_link.click()
        time.sleep(2)
    except Exception as e: 
        print('Не получилось перейти в сообщения!( Ошибка:')
        print(e)
        
def go_to_encounters():
    time.sleep(1)
    driver.refresh()
    time.sleep(2)

def get_new_messages():
    try:
        time.sleep(1)
        # текущий диалог
        dialogues = driver.find_elements_by_css_selector('.contact-card.im_user.js-contacts-item-content.is-current')
        # новые сообщения от 'старых' девочек
        dialogues += driver.find_elements_by_css_selector('.contact-card.has-new-message.im_user.js-contacts-item-content')
        # новый мэтчи
        dialogues += driver.find_elements_by_css_selector('.contact-card.is-new-match.im_user.js-contacts-item-content') 
        time.sleep(1)
        return list(dict.fromkeys(dialogues))
    except Exception as e: 
        print('Ошибка при получении диалогов!( Ошибка:')
        print(e)
    return []

def get_contacts():
    time.sleep(0.5)
    # получение контактов
    contacts = driver.find_elements_by_css_selector('.contacts__item.js-contacts-item')
    time.sleep(0.5)
    return contacts

def click_close():
    try:
        time.sleep(0.5)
        close_button = driver.find_element_by_css_selector('.ovl__close')
        close_button.click()
        time.sleep(0.5)
    except Exception as e: 
        a = 2 + 2
#         print('Не кликает кнопку закрыть!')
        #print(e)
    

def get_all_new_messages():
    try:
        dialogues = get_new_messages()
        new = len(dialogues)
        prev = 1
        while (new > prev):
            time.sleep(1.5)
            # переходим на последний доступный диалог на странице
            prev = new
#             print('Предыдущие:', prev)
            get_contacts()[-1].click()
            new_messages = get_new_messages()
            # сохраняем порядок, убираем дубликаты
            dialogues = list(dict.fromkeys(dialogues + new_messages))
            new = len(new_messages)

        return dialogues
    except Exception as e: 
        print('Ошибка при получении всех диалогов!( Ошибка:')
        print(e)
    return []

def reload_messages():
    time.sleep(1)
    driver.refresh()
    time.sleep(3)
    go_to_messages()

def reload_messages_and_like_n_girls(n):
    time.sleep(1)
    driver.refresh()
    time.sleep(15)
    new_button()
    like(n)
    time.sleep(3)
    go_to_messages()
    
def print_info():
    try:
        time.sleep(0.7)
        name = driver.find_element_by_css_selector('.connection-header__name')
        age = driver.find_element_by_css_selector('.connection-header__age')
        time.sleep(0.3)
        print(name.text + age.text)
        time.sleep(0.7)
    except Exception as e: 
        print('Ошибка при получении имени телочки!( Ошибка:')
        print(e)
    

def reply():
    global base
    try:
        txt_msgs = driver.find_elements_by_css_selector('.p-2.text-break-words')
        time.sleep(0.6)
        num_msgs = len(txt_msgs)
        if num_msgs == 0:
            send_message(s1)
        else:
            dialogue_level = 0
            flag_answered_first_msg = False
            flag_answered_second_msg = False
            messages_after_third_message = []
            for txt_msg in txt_msgs:
                if txt_msg.text in (s1, s2, s3):
                    dialogue_level += 1
                else:
                    if dialogue_level == 1:
                        flag_answered_first_msg = True
                    if dialogue_level == 2:
                        flag_answered_second_msg = True
                    elif dialogue_level == 3:
                        messages_after_third_message.append(txt_msg.text)
            if dialogue_level == 0:
                send_message(s1)
            elif dialogue_level == 1 and flag_answered_first_msg == True:
                send_message(s2)
            elif dialogue_level == 2 and flag_answered_second_msg == True:
                send_message(s3)
            elif len(messages_after_third_message) > 0:
                girls_contact = analyse_messages(messages_after_third_message)
                if girls_contact and girls_contact not in base:
                    base.add(girls_contact)
                    link = ''
                    try:
                        # ссылка на профиль девочки
                        girl_profile = driver.find_element_by_css_selector('.b-link.js-connection-profile')
                        link = girl_profile.get_attribute("href")
                        # убираем s из https://
                        link = 'http://' + link[8:]
                    except Exception as e: 
                        print('Ошибка при получении ссылки!( Ошибка:')
                        print(e)
                    print()
                    print_info()
                    print(girls_contact)
                    print(link)
                    print()
                
    
        time.sleep(0.3)
    except Exception as e: 
        print('Ошибка при ответе телочке!( Ошибка:')
        print(e)
    time.sleep(1)

login()

time.sleep(15)

new_button()

like(50)

go_to_messages()

while True:
    dialogues = get_all_new_messages()
    
    for dialogue in dialogues:
        flag_clicked_new_dialogue = True
        try:
            dialogue.click()
        except Exception as e: 
            #print(e)
            flag_clicked_new_dialogue = False
        time.sleep(1.5)
        if flag_clicked_new_dialogue:
            reply()
    
    number_of_minutes = 15
    print('- 0', end = ' - ')
    for i in range(100):
        time.sleep(60 * number_of_minutes / 100)
        #print('-- ' + str(100 * (i + 1) / number_of_minutes)  + '%' + ' --', end = '')
        if i % 5 == 0:
            print(str(i + 5), end = ' - ')
    print(end = '\n')
    reload_messages_and_like_n_girls(100)

time.sleep(10)

driver.quit()


Елизавета, 24
@elidistel
http://badoo.com/profile/0zAhMACTgxODU2MDE2Nwgu7lxkAAAAACDww7liHKwJXUjsnhV8tFXOSIZ2Ht-ozuCi_iUFKa7sKw?from=messenger

- 0 - 5 - 10 - 15 - 

In [2]:
print(base)
print(len(base))

{'kira_tesss', '89107201117', '89094638038', '@krpnkvlr', 'vitaliya_tishchenko', '89997801751'}
6


In [3]:
print(base)
print(len(base))

{'@oresama1208', '@marietta_03', 'malta_ss', 'vikkki2812', '89191381023', '@babka_masha', '@aleksandra199526', '@miss_ksenia_12', '89854655730', '@pvostrukhina', 'https://tme/tinatin99', '79168681468', '@delfon', '89265401224', '@ustinatasha', '@anna_surowaja', '@ksenia_gribanova', 'karachka2020', '89161949858', '89773137747', '@tereshonok046'}
21


In [8]:
print(base)
print(len(base))

{'@salashinskaya', '@belykh_p', '89623348361', 'sure', 'belova8967', '@stephani0', '89166319614', '@eka_sham', '@evgeniya1892', 'kata_6_6', 'alinkaa_meow', '89777415662', '89183962253', 'yanchik_d', 'vi_aviii', '89604511816', 'jpjp39', '@eirojr', 'fox_58_fox', '@alina11211024', '89511202134', 'https://tme/webdesignbsk', '@vi3sa', 'viiiere', '89266605270', 'https://tme/isabellabelka', 'kitikill', 'i', '@annageiden', '89150170718', '89056787477', '@varechka123', '@jwxt_jwxt'}
33


In [3]:
print(base)
print(len(base))

{'bless_exc', '@val120595', '89998576674', '@yu_selfish', '@victoriavenski', 'https://tme/npavlovna', 'kats69', '89029495459', '@innnakentiy', '@ginger_natali', 'elvmh', '@svetlana6514', '89094324439', '89523599884', 'dasha_367', '@jesuisli', '89998731423', '89376583394', '89772927098', '89637768129', '@penguinchik1'}
21


In [4]:
print(base)
print(len(base))

{'mtnsva', 'mn93_h', 'margo_kumm', '@alshvetsova', '89144506712', '89523519221', 'whatsapp', 'dashik_30', 'nem_arda', '8911409809', 'refipos', '@palmind99', '89956577388', 'https://tme/kathlee_k', '@dkomarova21', 'unforgetablle17', '@amelia_389', '@elenadamaskina'}
18


In [5]:
print(base)
print(len(base))

{'@wryyyyy0', 'lova_alinka', 'vanniesss', '89381315882', '89614308812', '@princessdi15', '79613047814', '79896131773', 'prostite_shto', '89194700608', '@quheondar', 'vikaflora8', 'bk_192', '89888995270', '@l_kseniia_l', '89112273560', 'pilipaerkatena', '@leslii', '89515179031', 'slushaitelyudi', '@yliuakot27', '89215736389', '89507558475', '89273568274'}
24


In [3]:
print(base)
print(len(base))

{'kozlitina_25', '@yliapomanovna', '89044427796', '@sandrak_22', '89524148925', '@dasha_eremina', '@sema_kosh', '@popkorn007', '@d_solovei', '@katty_2106', '@spassibaattika', '89615810171', '89188501252', 'skm_love', 'libertystates', '89515387474', '89649318432', '79996998426', 'khorkovayu', 'https://tme/radoniz', '89064230588', 'https://tme/li_anova', '89614078688', 'daryadasany', '@tulusheva', '@dashashulman'}
26


In [2]:
print(base)
print(len(base))

{'@lordofmordor', '89281665928', '89896122964', 'margomargo_s', '89034728782', '9054264464', '89613244246', '89518234857', '@ornelllo', 'ddau42', '@dj_jenny', '@kira_avedova'}
12


In [13]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import warnings
import string

warnings.filterwarnings("ignore")
s = Service('/Users/maksimveretennikov/Desktop/Coding/Project_Badoo/chromedriver')
driver = webdriver.Chrome(service = s)
driver.get('https://badoo.com/en-us/signin/?f=top')

n_likes = 0

def login():
    try:
        login_field = driver.find_element_by_css_selector('.js-signin-login')
        login_field.send_keys('veretennikov.maksi@yandex.ru')
        pass_field = driver.find_element_by_css_selector('.js-signin-password')
        pass_field.send_keys('192837465m')
        time.sleep(2)
        btn_login = driver.find_element_by_css_selector('.btn--block')
        btn_login.click()
        time.sleep(6)
    except Exception as e: 
        print('Не логинится( Ошибка:')
        print(e)
    
def skip():
    try:
        time.sleep(1)
        btn_skip = driver.find_element_by_css_selector('.js-chrome-pushes-deny')
        btn_skip.click()
        time.sleep(1)
    except Exception as e: 
        print('Не пропускает сообщение( Ошибка:')
        print(e)

def like(n):
    global n_likes
    
    for i in range(n):
        try:
            time.sleep(0.2)
            btn_like = driver.find_element_by_css_selector('.js-profile-header-vote-yes')
            btn_like.click()

            if n_likes == 0:
                skip()
            n_likes += 1
        except Exception as e: 
            print('Не лайкает( Ошибка:')
            print(e) 
            break
    
login()
    
like(500)     

print(n_likes)





time.sleep(30)


500


In [6]:
a = [1, 2, 3]
print(a[-1])

3


In [17]:
import yapf.yapflib.yapf_api

In [5]:
a = [2, 3, 4]
b = [4, 6, 5]
a = list(set(a + b))
print(a[-1])

6


In [4]:
import time
for i in range(5):
    print(str(100 * (i + 1) / 5 )  + '%', 'has passed', end = ' ')
    time.sleep(2)

20.0% has passed 40.0% has passed 60.0% has passed 80.0% has passed 100.0% has passed 

In [6]:
print('a', end = '')
print(end = '\n')
print('c')
print('d')

a
c
d


In [2]:
items = [1, 3, 0, 1, 3, 2]
list(dict.fromkeys(items))

[1, 3, 0, 2]

In [ ]:
import signal  
import time  

# Our signal handler
def signal_handler(signum, frame):  
    print("Signal Number:", signum, " Frame: ", frame)  

def exit_handler(signum, frame):
    print('Exiting....')
    exit(0)

# Register our signal handler with `SIGINT`(CTRL + C)
signal.signal(signal.SIGINT, signal_handler)

# Register the exit handler with `SIGTSTP` (Ctrl + Z)
signal.signal(signal.SIGTSTP, exit_handler)

# While Loop
while 1:  
    print("Press Ctrl + C") 
    time.sleep(3) 

Press Ctrl + C
Press Ctrl + C
Press Ctrl + C
Signal Number: 2  Frame:  <frame at 0x7fee13dc98c0, file '<ipython-input-1-5d93e63d2c25>', line 21, code <module>>
Press Ctrl + C
Press Ctrl + C
Press Ctrl + C


In [ ]:
base = set()
base.add('aa')
print(base)
a = 'https://badoo.com/profile/0zAhMACTkyMzI4NDIxMQgu7lxkAAAAACDdUtRSiJ2jiLIgwEROMWvqP2iiYXfDudCOI3coT8cFqQ?from=messenger'
print(a[5:])